# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f4848c60c70>
├── 'a' --> tensor([[ 0.8454,  1.5214, -0.4193],
│                   [-0.2736,  0.1687, -0.3490]])
└── 'x' --> <FastTreeValue 0x7f4848c60d00>
    └── 'c' --> tensor([[ 0.2995, -0.5976,  1.4644,  0.5437],
                        [-0.7744,  1.3820,  1.8354,  1.4022],
                        [-0.3931,  2.4337,  0.6316,  2.2762]])

In [4]:
t.a

tensor([[ 0.8454,  1.5214, -0.4193],
        [-0.2736,  0.1687, -0.3490]])

In [5]:
%timeit t.a

58.4 ns ± 0.1 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f4848c60c70>
├── 'a' --> tensor([[-0.8341, -0.8733, -0.9067],
│                   [-0.0694, -1.1300,  0.6341]])
└── 'x' --> <FastTreeValue 0x7f4848c60d00>
    └── 'c' --> tensor([[ 0.2995, -0.5976,  1.4644,  0.5437],
                        [-0.7744,  1.3820,  1.8354,  1.4022],
                        [-0.3931,  2.4337,  0.6316,  2.2762]])

In [7]:
%timeit t.a = new_value

58.6 ns ± 0.0612 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.8454,  1.5214, -0.4193],
               [-0.2736,  0.1687, -0.3490]]),
    x: Batch(
           c: tensor([[ 0.2995, -0.5976,  1.4644,  0.5437],
                      [-0.7744,  1.3820,  1.8354,  1.4022],
                      [-0.3931,  2.4337,  0.6316,  2.2762]]),
       ),
)

In [10]:
b.a

tensor([[ 0.8454,  1.5214, -0.4193],
        [-0.2736,  0.1687, -0.3490]])

In [11]:
%timeit b.a

54.5 ns ± 0.0485 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.5385, -0.4130, -0.6989],
               [ 0.2925,  0.9348,  1.2033]]),
    x: Batch(
           c: tensor([[ 0.2995, -0.5976,  1.4644,  0.5437],
                      [-0.7744,  1.3820,  1.8354,  1.4022],
                      [-0.3931,  2.4337,  0.6316,  2.2762]]),
       ),
)

In [13]:
%timeit b.a = new_value

485 ns ± 0.059 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

841 ns ± 8.39 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.6 µs ± 20.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

135 µs ± 501 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 1.38 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f4791f6bb20>
├── 'a' --> tensor([[[ 0.8454,  1.5214, -0.4193],
│                    [-0.2736,  0.1687, -0.3490]],
│           
│                   [[ 0.8454,  1.5214, -0.4193],
│                    [-0.2736,  0.1687, -0.3490]],
│           
│                   [[ 0.8454,  1.5214, -0.4193],
│                    [-0.2736,  0.1687, -0.3490]],
│           
│                   [[ 0.8454,  1.5214, -0.4193],
│                    [-0.2736,  0.1687, -0.3490]],
│           
│                   [[ 0.8454,  1.5214, -0.4193],
│                    [-0.2736,  0.1687, -0.3490]],
│           
│                   [[ 0.8454,  1.5214, -0.4193],
│                    [-0.2736,  0.1687, -0.3490]],
│           
│                   [[ 0.8454,  1.5214, -0.4193],
│                    [-0.2736,  0.1687, -0.3490]],
│           
│                   [[ 0.8454,  1.5214, -0.4193],
│                    [-0.2736,  0.1687, -0.3490]]])
└── 'x' --> <FastTreeValue 0x7f4791f6bac0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.6 µs ± 101 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f4791f6be80>
├── 'a' --> tensor([[ 0.8454,  1.5214, -0.4193],
│                   [-0.2736,  0.1687, -0.3490],
│                   [ 0.8454,  1.5214, -0.4193],
│                   [-0.2736,  0.1687, -0.3490],
│                   [ 0.8454,  1.5214, -0.4193],
│                   [-0.2736,  0.1687, -0.3490],
│                   [ 0.8454,  1.5214, -0.4193],
│                   [-0.2736,  0.1687, -0.3490],
│                   [ 0.8454,  1.5214, -0.4193],
│                   [-0.2736,  0.1687, -0.3490],
│                   [ 0.8454,  1.5214, -0.4193],
│                   [-0.2736,  0.1687, -0.3490],
│                   [ 0.8454,  1.5214, -0.4193],
│                   [-0.2736,  0.1687, -0.3490],
│                   [ 0.8454,  1.5214, -0.4193],
│                   [-0.2736,  0.1687, -0.3490]])
└── 'x' --> <FastTreeValue 0x7f4848c913a0>
    └── 'c' --> tensor([[ 0.2995, -0.5976,  1.4644,  0.5437],
                        [-0.7744,  1.3820,  1.8354,  1.4022],
                 

In [23]:
%timeit t_cat(trees)

29.9 µs ± 70.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59.3 µs ± 203 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.8454,  1.5214, -0.4193],
                [-0.2736,  0.1687, -0.3490]],
       
               [[ 0.8454,  1.5214, -0.4193],
                [-0.2736,  0.1687, -0.3490]],
       
               [[ 0.8454,  1.5214, -0.4193],
                [-0.2736,  0.1687, -0.3490]],
       
               [[ 0.8454,  1.5214, -0.4193],
                [-0.2736,  0.1687, -0.3490]],
       
               [[ 0.8454,  1.5214, -0.4193],
                [-0.2736,  0.1687, -0.3490]],
       
               [[ 0.8454,  1.5214, -0.4193],
                [-0.2736,  0.1687, -0.3490]],
       
               [[ 0.8454,  1.5214, -0.4193],
                [-0.2736,  0.1687, -0.3490]],
       
               [[ 0.8454,  1.5214, -0.4193],
                [-0.2736,  0.1687, -0.3490]]]),
    x: Batch(
           c: tensor([[[ 0.2995, -0.5976,  1.4644,  0.5437],
                       [-0.7744,  1.3820,  1.8354,  1.4022],
                       [-0.3931,  2.4337,  0.6316,  2.2762]],
         

In [26]:
%timeit Batch.stack(batches)

77.1 µs ± 102 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.8454,  1.5214, -0.4193],
               [-0.2736,  0.1687, -0.3490],
               [ 0.8454,  1.5214, -0.4193],
               [-0.2736,  0.1687, -0.3490],
               [ 0.8454,  1.5214, -0.4193],
               [-0.2736,  0.1687, -0.3490],
               [ 0.8454,  1.5214, -0.4193],
               [-0.2736,  0.1687, -0.3490],
               [ 0.8454,  1.5214, -0.4193],
               [-0.2736,  0.1687, -0.3490],
               [ 0.8454,  1.5214, -0.4193],
               [-0.2736,  0.1687, -0.3490],
               [ 0.8454,  1.5214, -0.4193],
               [-0.2736,  0.1687, -0.3490],
               [ 0.8454,  1.5214, -0.4193],
               [-0.2736,  0.1687, -0.3490]]),
    x: Batch(
           c: tensor([[ 0.2995, -0.5976,  1.4644,  0.5437],
                      [-0.7744,  1.3820,  1.8354,  1.4022],
                      [-0.3931,  2.4337,  0.6316,  2.2762],
                      [ 0.2995, -0.5976,  1.4644,  0.5437],
                      [-0.7744,  

In [28]:
%timeit Batch.cat(batches)

143 µs ± 319 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

318 µs ± 858 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
